In [1]:
!pip install qiskit==0.43.0
!pip install qiskit-machine-learning==0.5.0
!pip install imbalanced-learn

In [2]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import accuracy_score, classification_report
from sklearn.svm import SVC
from imblearn.over_sampling import SMOTE

from qiskit import Aer
from qiskit.utils import QuantumInstance
from qiskit.circuit.library import PauliFeatureMap
from qiskit_machine_learning.kernels import QuantumKernel

In [5]:
df = pd.read_csv('/content/sample_data/anemia.csv')
df = df.dropna()
X = df[["Hemoglobin", "MCHC"]].values  # Using top 2 features only
y = df["Result"].values

In [6]:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
y = le.fit_transform(y)

In [7]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [8]:
scaler = MinMaxScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

In [9]:
sm = SMOTE(random_state=42)
X_train, y_train = sm.fit_resample(X_train, y_train)

/usr/local/lib/python3.11/dist-packages/sklearn/base.py:474: FutureWarning: `BaseEstimator._validate_data` is deprecated in 1.6 and will be removed in 1.7. Use `sklearn.utils.validation.validate_data` instead. This function becomes public and is part of the scikit-learn developer API.
  warnings.warn(


In [11]:
feature_dim = X_train.shape[1]
feature_map = PauliFeatureMap(feature_dimension=feature_dim, reps=3, entanglement='full', paulis=['X', 'Y', 'Z'])
quantum_instance = QuantumInstance(backend=Aer.get_backend("statevector_simulator"))
quantum_kernel = QuantumKernel(feature_map=feature_map, quantum_instance=quantum_instance)

/tmp/ipython-input-11-1576946681.py:3: DeprecationWarning: The class ``qiskit.utils.quantum_instance.QuantumInstance`` is deprecated as of qiskit-terra 0.24.0. It will be removed no earlier than 3 months after the release date. For code migration guidelines, visit https://qisk.it/qi_migration.
  quantum_instance = QuantumInstance(backend=Aer.get_backend("statevector_simulator"))


In [12]:
kernel_train = quantum_kernel.evaluate(x_vec=X_train)
kernel_test = quantum_kernel.evaluate(x_vec=X_test, y_vec=X_train)

In [13]:
model = SVC(kernel='precomputed')
model.fit(kernel_train, y_train)


SVC(kernel='precomputed')

In [14]:
y_pred = model.predict(kernel_test)
print("\n🔮 Quantum SVM Accuracy:", accuracy_score(y_test, y_pred))
print("\n📊 Classification Report:\n", classification_report(y_test, y_pred))


🔮 Quantum SVM Accuracy: 0.9368421052631579

📊 Classification Report:
               precision    recall  f1-score   support

           0       0.97      0.92      0.94       157
           1       0.90      0.96      0.93       128

    accuracy                           0.94       285
   macro avg       0.94      0.94      0.94       285
weighted avg       0.94      0.94      0.94       285



In [15]:
# ✅ Get input from user
try:
    hemoglobin = float(input("Enter Hemoglobin value: "))
    mchc = float(input("Enter MCHC value: "))

    # Normalize the input using the same scaler used for training
    user_input = scaler.transform([[hemoglobin, mchc]])

    # Compute quantum kernel between input and training data
    user_kernel = quantum_kernel.evaluate(x_vec=user_input, y_vec=X_train)

    # Predict using trained model
    user_prediction = model.predict(user_kernel)

    # Decode label if encoded
    predicted_label = le.inverse_transform(user_prediction)

    print(f"\n🔮 Predicted Result: {predicted_label[0]} No Anemic")

except Exception as e:
    print("❌ Invalid input or error occurred:", e)


Enter Hemoglobin value: 14.9
Enter MCHC value: 29.1

🔮 Predicted Result: 0
